In [9]:
fraction=0.3

In [1]:
%run Setup.ipynb

In [2]:
df = pd.read_csv(DATA_PATH+'/df.csv')
df=df.drop('Unnamed: 0',axis=1)

In [4]:
train_corpus, test_corpus, Y_train, Y_test =\
                                 train_test_split(np.array(df['CCN clean']), np.array(df['True Product']), test_size=0.1, random_state=42)

train_corpus.shape, test_corpus.shape

((18000,), (2000,))

In [5]:
tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0)
tv_train_features = tv.fit_transform(train_corpus)
tv_test_features = tv.transform(test_corpus)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

TFIDF model:> Train features shape: (18000, 24379)  Test features shape: (2000, 24379)


Random

In [6]:
vec=[]
for phrase in test_corpus:
    vec.append(phrase)

In [7]:
listword=[]
for phrase in vec:
    for word in phrase.split():
        listword.append(word)

In [10]:
n_srch=round(fraction*len(listword))

In [11]:
to_rep_rand=random.sample(listword,n_srch)
to_rep_rand
replaced=to_rep_rand.copy()

Random Glove

In [11]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [12]:
lol_w2v_test=[]
for i in range(len(test_corpus)):
    lol_w2v_test.append(nltk.word_tokenize(test_corpus[i]))

In [13]:
new_test_rand_glove=test_corpus.copy()

In [29]:
for i in range(len(lol_w2v_test)):
    n_srch=round(fraction*len(lol_w2v_test[i]))
    to_rep_rand=random.sample(lol_w2v_test[i],n_srch)
    rand_sims=[]
    replaced_rand_glove=[]
    for j in range(len(to_rep_rand)):
        appoggio_rand=to_rep_rand[j]
        try:
            rand_sims.append(glove_vectors.most_similar(appoggio_rand,topn=1))
        except Exception:
            rand_sims.append([(appoggio_rand,1)])
        replaced_rand_glove.append(list(rand_sims[j][0])[0])
    for f in range(len(replaced_rand_glove)):
        try:
            new_test_rand_glove[i] = new_test_rand_glove[i].replace(to_rep_rand[f],replaced_rand_glove[f],1)
        except Exception:
                pass
    

In [30]:
tv_test_features_rand_glove = tv.transform(new_test_rand_glove)

In [50]:
pd.DataFrame(tv_test_features_rand_glove).to_csv(RESULTS_PATH+'/test_features_rand_glove_3.csv')

In [18]:
svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(tv_train_features, Y_train)
svm_random_3 = svm.score(tv_test_features, Y_test)
print('Test Accuracy:', svm_random_3)

Test Accuracy: 0.8355


In [43]:
xgboost = xgb.XGBClassifier(min_child_weight=1,
                                max_depth=6,
                                subsample=1,
                                colsample_bytree=1,random_state=42)
xgboost.fit(tv_train_features, Y_train)
xgboost_random_3 = xgboost.score(tv_test_features_rand_glove, Y_test)
print('Test Accuracy:', xgboost_random_3)

/Users/giorgiasacco/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:26:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test Accuracy: 0.795


In [47]:
Random_xgboost_svm_3=[xgboost_random_3,svm_random_3]
Random_xgboost_svm_3

[0.795, 0.7735]

In [49]:
pd.DataFrame(Random_xgboost_svm_3).to_csv(RESULTS_PATH+'/Random_xgboost_svm_3.csv')

TF IDF

In [33]:
list1 = test_corpus.tolist()

In [34]:
for i in range(len(list1)):
    list1[i]=tb(list1[i])

In [35]:
num_per_doc=[]
for i in range(len(test_corpus)):
    appoggio=[]
    for word in test_corpus[i].split():
        appoggio.append(word)
    num_per_doc.append(len(appoggio))

In [36]:
new_test_tf_glove=test_corpus.copy()

In [37]:
for i, blob in enumerate(list1):
    counter=0
    to_rep_tf=[]
    scores = {word: tfidf(word, blob, list1) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for s in range(round(num_per_doc[i]*fraction)):
        try:
            to_rep_tf.append(list(sorted_words[s])[0])
        except Exception:
            break
    tf_sims=[]
    for word in range(len(to_rep_tf)):
        appoggio=to_rep_tf[word]
        try:
            tf_sims.append(glove_vectors.most_similar(appoggio,topn=1))
        except Exception:
            tf_sims.append([(appoggio,1)])
    replaced_tf_glove=[]
    for word in range(len(tf_sims)):
        replaced_tf_glove.append(list(tf_sims[word][0])[0])
    for j in range(len(replaced_tf_glove)):
        a=0
        counter_same_word=0
        if to_rep_tf[j]==replaced_tf_glove[j]:
            pass
        while a!=1:
            if to_rep_tf[j] in new_test_tf_glove[i]:
                new_test_tf_glove[i] = new_test_tf_glove[i].replace(to_rep_tf[j],replaced_tf_glove[j],1)
                counter+=1
                counter_same_word+=1
            else: 
                a=1
            if counter_same_word==5:
                break
        if counter== round(num_per_doc[i]*fraction):
            break

In [38]:
tv_test_features_tf_glove = tv.transform(new_test_tf_glove)

In [39]:
pd.DataFrame(tv_test_features_tf_glove).to_csv(RESULTS_PATH+'/test_features_tfidf_3.csv')

In [40]:
svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(tv_train_features, Y_train)
svm_tfidf_3 = svm.score(tv_test_features_tf_glove, Y_test)
print('Test Accuracy:', svm_tfidf_3) 
#C

Test Accuracy: 0.6355


In [41]:
xgboost = xgb.XGBClassifier(min_child_weight=1,
                                max_depth=6,
                                subsample=1,
                                colsample_bytree=1, random_state=42)
xgboost.fit(tv_train_features, Y_train)
xgboost_tfidf_3 = xgboost.score(tv_test_features_tf_glove, Y_test)
print('Test Accuracy:', xgboost_tfidf_3)

C:\Users\danie\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:38:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test Accuracy: 0.721


In [42]:
tfidf_xgboost_svm_3=[xgboost_tfidf_3,svm_tfidf_3]
tfidf_xgboost_svm_3

[0.721, 0.6355]

In [43]:
pd.DataFrame(tfidf_xgboost_svm_3).to_csv(RESULTS_PATH+'/tfidf_xgboost_svm_3.csv')

Weight based

In [20]:
y = svm.decision_function(tv_test_features)
w_norm = np.linalg.norm(svm.coef_)
dist = y / w_norm

In [21]:
distances= []
for i in range(len(dist)):
    distances.append(sqdist(dist[i]))

In [22]:
pd_dist=pd.Series(distances)
sorted_pd=pd_dist.sort_values()

In [23]:
indexes=[]
for i in range(round(len(sorted_pd)*fraction)):
    indexes.append(list(sorted_pd.index)[i])

In [24]:
vec=[]
for i in range(len(indexes)):
    vec.append(test_corpus[indexes[i]])

In [25]:
new_test_weight_glove=test_corpus.copy()

In [26]:
for i in range(len(vec)):
    to_rep_weight=[]
    weight_sims=[]
    for word in vec[i].split():
        to_rep_weight.append(word)
    replaced_weight_glove=[]
    for j in range(len(to_rep_weight)):
        appoggio=to_rep_weight[j]
        try:
            weight_sims.append(glove_vectors.most_similar(appoggio,topn=1))
        except Exception:
            weight_sims.append([(appoggio,1)])
        replaced_weight_glove.append(list(weight_sims[j][0])[0])
    for j in range(len(replaced_weight_glove)):
        if to_rep_weight[j]==replaced_weight_glove[j]:
            pass
        try:
            new_test_weight_glove[indexes[i]] = listToString(replaced_weight_glove)
        except Exception:
            pass

In [27]:
tv_test_features_weight_glove = tv.transform(new_test_weight_glove)

In [28]:
pd.DataFrame(tv_test_features_weight_glove).to_csv(RESULTS_PATH+'/test_features_weight_3.csv')

In [29]:
svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(tv_train_features, Y_train)
svm_weight_3 = svm.score(tv_test_features_weight_glove, Y_test)
print('Test Accuracy:', svm_weight_3)

Test Accuracy: 0.8355


In [30]:
xgboost = xgb.XGBClassifier(min_child_weight=1,
                                max_depth=6,
                                subsample=1,
                                colsample_bytree=1, random_state=42)
xgboost.fit(tv_train_features, Y_train)
xgboost_weight_3 = xgboost.score(tv_test_features_weight_glove, Y_test)
print('Test Accuracy:', xgboost_weight_3)

C:\Users\danie\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:08:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Test Accuracy: 0.8345


In [31]:
weight_xgboost_svm_3=[xgboost_weight_3,svm_weight_3]
weight_xgboost_svm_3

[0.8345, 0.8355]

In [32]:
pd.DataFrame(weight_xgboost_svm_3).to_csv(RESULTS_PATH+'/weight_xgboost_svm_3.csv')